# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
!pip3 install xlsxwriter  

     |████████████████████████████████| 149 kB 24.9 MB/s 


In [2]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 128623699,
 'calculationPrice': 'close',
 'change': -1.15,
 'changePercent': -0.0065,
 'close': 177.98,
 'closeSource': 'flofiaci',
 'closeTime': 1664530424127,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': 171.38,
 'delayedPriceTime': 1685321637302,
 'extendedChange': 0.77,
 'extendedChangePercent': 0.00459,
 'extendedPrice': 177.85,
 'extendedPriceTime': 1663077575723,
 'high': 182.02,
 'highSource': 'u cm 5eddreyelnitaie p1',
 'highTime': 1642033988549,
 'iexAskPrice': None,
 'iexAskSize': None,
 'iexBidPrice': None,
 'iexBidSize': None,
 'iexClose': 172.23,
 'iexCloseTime': 1706526064558,
 'iexLastUpdated': None,
 'iexMarketPercent': None,
 'iexOpen': 172.93,
 'iexOpenTime': 1655499532585,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexVolume': None,
 'isUSMarketOpen': False,
 'lastTradeTime': 1715453289207,
 'latestPrice': 178.18,
 'latestSource': 'Close',
 'latestTime': 'December 17, 2021',
 'latestUpdate': 1646391677980,
 'latest

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [5]:
pe_ratio = data['peRatio']
pe_ratio

15.6

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [6]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [7]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
      if symbol!='KSU':
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,158.42,48.84,N/A
1,AAL,17.30,-3.49,N/A
2,AAP,242.39,24.67,N/A
3,AAPL,172.07,15.88,N/A
4,ABBV,135.97,32.22,N/A
...,...,...,...,...
499,YUM,136.74,25.79,N/A
500,ZBH,124.45,31.31,N/A
501,ZBRA,612.39,38.08,N/A
502,ZION,62.50,5.88,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [8]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(drop=True, inplace = True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,HPE,15.39,2.95,N/A
1,BIO,743.72,3.38,N/A
2,COO,412.39,3.48,N/A
3,EBAY,65.62,3.5,N/A
4,HPQ,37.34,3.52,N/A
5,NRG,41.18,4.23,N/A
6,MPC,64.97,4.58,N/A
7,DHI,106.04,4.66,N/A
8,BEN,34.65,4.79,N/A
9,TSN,89.30,5.24,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [9]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [10]:
portfolio_input()

Enter the value of your portfolio:1000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [11]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,HPE,15.39,2.95,1299
1,BIO,743.72,3.38,26
2,COO,412.39,3.48,48
3,EBAY,65.62,3.5,304
4,HPQ,37.34,3.52,535
5,NRG,41.18,4.23,485
6,MPC,64.97,4.58,307
7,DHI,106.04,4.66,188
8,BEN,34.65,4.79,577
9,TSN,89.30,5.24,223


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [64]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()
 
# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit


Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [65]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)


In [66]:
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )
rv_dataframe      

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,154.34,N/A,48.17,N/A,9.62,N/A,7.65,N/A,30.754467,N/A,14.603702,N/A,N/A
1,AAL,17.42,N/A,-3.4,N/A,-1.54,N/A,0.4678,N/A,-7.180007,N/A,1.483191,N/A,N/A
2,AAP,236.91,N/A,24.12,N/A,4.66,N/A,1.37,N/A,13.269616,N/A,3.018627,N/A,N/A
3,AAPL,172.23,N/A,15.5,N/A,22.57,N/A,3.87,N/A,12.283568,N/A,9.533892,N/A,N/A
4,ABBV,135.91,N/A,31.85,N/A,17.54,N/A,4.22,N/A,11.128174,N/A,8.054098,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,134.79,N/A,26.01,N/A,-4.93,N/A,6.13,N/A,21.050402,N/A,10.052231,N/A,N/A
501,ZBH,121.55,N/A,31.57,N/A,2.08,N/A,3.3,N/A,18.283760,N/A,5.774630,N/A,N/A
502,ZBRA,611.62,N/A,39.4,N/A,11.52,N/A,6.03,N/A,28.124258,N/A,12.515446,N/A,N/A
503,ZION,63.40,N/A,6.04,N/A,1.32,N/A,2.82,N/A,4.607352,N/A,2.455285,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [67]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,190.600,N/A,61.16,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,301.710,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,296.010,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,12.000,N/A,9.98,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,23.450,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,50.040,N/A,14.91,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
186,FLIR,59.210,N/A,32.24,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,35.360,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,2897.610,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
325,MXIM,105.310,N/A,34.17,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [68]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [69]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [70]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

0       0.819802
1      0.0524752
2        0.49703
3        0.30297
4       0.645545
         ...    
500     0.548515
501     0.641584
502     0.750495
503    0.0950495
504     0.887129
Name: PE Percentile, Length: 505, dtype: object
0       0.807921
1      0.0554455
2        0.60198
3       0.938614
4       0.916832
         ...    
500    0.0514851
501      0.29901
502     0.839604
503     0.148515
504     0.944554
Name: PB Percentile, Length: 505, dtype: object
0       0.823762
1      0.0475248
2        0.20198
3       0.576238
4       0.617822
         ...    
500     0.764356
501     0.516832
502     0.758416
503     0.450495
504     0.934653
Name: PS Percentile, Length: 505, dtype: object
0       0.841584
1      0.0237624
2       0.441584
3        0.39802
4       0.350495
         ...    
500      0.70297
501     0.627723
502     0.813861
503    0.0653465
504     0.936634
Name: EV/EBITDA Percentile, Length: 505, dtype: object
0       0.853465
1      0.0673267
2       0.174257
3 

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,154.34,N/A,48.17,0.819802,9.62,0.807921,7.6500,0.823762,30.754467,0.841584,14.603702,0.853465,N/A
1,AAL,17.42,N/A,-3.40,0.0524752,-1.54,0.0554455,0.4678,0.0475248,-7.180007,0.0237624,1.483191,0.0673267,N/A
2,AAP,236.91,N/A,24.12,0.49703,4.66,0.60198,1.3700,0.20198,13.269616,0.441584,3.018627,0.174257,N/A
3,AAPL,172.23,N/A,15.50,0.30297,22.57,0.938614,3.8700,0.576238,12.283568,0.39802,9.533892,0.667327,N/A
4,ABBV,135.91,N/A,31.85,0.645545,17.54,0.916832,4.2200,0.617822,11.128174,0.350495,8.054098,0.576238,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,134.79,N/A,26.01,0.548515,-4.93,0.0514851,6.1300,0.764356,21.050402,0.70297,10.052231,0.69703,N/A
501,ZBH,121.55,N/A,31.57,0.641584,2.08,0.29901,3.3000,0.516832,18.283760,0.627723,5.774630,0.386139,N/A
502,ZBRA,611.62,N/A,39.40,0.750495,11.52,0.839604,6.0300,0.758416,28.124258,0.813861,12.515446,0.786139,N/A
503,ZION,63.40,N/A,6.04,0.0950495,1.32,0.148515,2.8200,0.450495,4.607352,0.0653465,2.455285,0.126733,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [71]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,154.34,N/A,48.17,0.819802,9.62,0.807921,7.6500,0.823762,30.754467,0.841584,14.603702,0.853465,0.829307
1,AAL,17.42,N/A,-3.40,0.0524752,-1.54,0.0554455,0.4678,0.0475248,-7.180007,0.0237624,1.483191,0.0673267,0.0493069
2,AAP,236.91,N/A,24.12,0.49703,4.66,0.60198,1.3700,0.20198,13.269616,0.441584,3.018627,0.174257,0.383366
3,AAPL,172.23,N/A,15.50,0.30297,22.57,0.938614,3.8700,0.576238,12.283568,0.39802,9.533892,0.667327,0.576634
4,ABBV,135.91,N/A,31.85,0.645545,17.54,0.916832,4.2200,0.617822,11.128174,0.350495,8.054098,0.576238,0.621386
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,134.79,N/A,26.01,0.548515,-4.93,0.0514851,6.1300,0.764356,21.050402,0.70297,10.052231,0.69703,0.552871
501,ZBH,121.55,N/A,31.57,0.641584,2.08,0.29901,3.3000,0.516832,18.283760,0.627723,5.774630,0.386139,0.494257
502,ZBRA,611.62,N/A,39.40,0.750495,11.52,0.839604,6.0300,0.758416,28.124258,0.813861,12.515446,0.786139,0.789703
503,ZION,63.40,N/A,6.04,0.0950495,1.32,0.148515,2.8200,0.450495,4.607352,0.0653465,2.455285,0.126733,0.177228


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [72]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,24.70,N/A,6.10,0.0970297,0.4548,0.0574257,0.3815,0.0316832,2.908011,0.0316832,0.361499,0.00792079,0.0451485
1,AAL,17.42,N/A,-3.40,0.0524752,-1.5400,0.0554455,0.4678,0.0475248,-7.180007,0.0237624,1.483191,0.0673267,0.0493069
2,HPQ,38.03,N/A,3.46,0.0673267,-13.0000,0.0356436,0.3151,0.0138614,3.971091,0.049505,1.659744,0.0851485,0.050297
3,PRU,108.71,N/A,5.89,0.0930693,0.6745,0.0673267,0.5892,0.0574257,4.467762,0.0613861,0.556841,0.00990099,0.0578218
4,BEN,34.06,N/A,4.82,0.0792079,0.7498,0.0772277,1.0264,0.136634,3.163926,0.0356436,0.813679,0.019802,0.069703
5,KSS,49.85,N/A,7.50,0.120792,1.3700,0.157426,0.3625,0.029703,3.867316,0.0455446,1.159317,0.049505,0.0805941
6,BA,195.70,N/A,-13.69,0.0415842,-8.1100,0.0475248,1.9000,0.310891,-32.144558,0.00594059,-1272.516069,0.0019802,0.0815842
7,HPE,15.18,N/A,3.06,0.0633663,0.5009,0.0594059,0.3580,0.0277228,7.343353,0.160396,2.033914,0.108911,0.0839604
8,MET,62.93,N/A,10.14,0.176238,0.7700,0.0792079,0.7261,0.0811881,4.840542,0.0732673,0.712044,0.0178218,0.0855446
9,AIG,54.72,N/A,8.82,0.142574,0.7061,0.0712871,0.9868,0.126733,4.267262,0.0574257,0.968127,0.0316832,0.0859406


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [73]:
portfolio_input()

Enter the value of your portfolio:2500000


In [74]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,24.70,2024,6.10,0.0970297,0.4548,0.0574257,0.3815,0.0316832,2.908011,0.0316832,0.361499,0.00792079,0.0451485
1,AAL,17.42,2870,-3.40,0.0524752,-1.5400,0.0554455,0.4678,0.0475248,-7.180007,0.0237624,1.483191,0.0673267,0.0493069
2,HPQ,38.03,1314,3.46,0.0673267,-13.0000,0.0356436,0.3151,0.0138614,3.971091,0.049505,1.659744,0.0851485,0.050297
3,PRU,108.71,459,5.89,0.0930693,0.6745,0.0673267,0.5892,0.0574257,4.467762,0.0613861,0.556841,0.00990099,0.0578218
4,BEN,34.06,1467,4.82,0.0792079,0.7498,0.0772277,1.0264,0.136634,3.163926,0.0356436,0.813679,0.019802,0.069703
5,KSS,49.85,1003,7.50,0.120792,1.3700,0.157426,0.3625,0.029703,3.867316,0.0455446,1.159317,0.049505,0.0805941
6,BA,195.70,255,-13.69,0.0415842,-8.1100,0.0475248,1.9000,0.310891,-32.144558,0.00594059,-1272.516069,0.0019802,0.0815842
7,HPE,15.18,3293,3.06,0.0633663,0.5009,0.0594059,0.3580,0.0277228,7.343353,0.160396,2.033914,0.108911,0.0839604
8,MET,62.93,794,10.14,0.176238,0.7700,0.0792079,0.7261,0.0811881,4.840542,0.0732673,0.712044,0.0178218,0.0855446
9,AIG,54.72,913,8.82,0.142574,0.7061,0.0712871,0.9868,0.126733,4.267262,0.0574257,0.968127,0.0316832,0.0859406


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [79]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [80]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [81]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [82]:
writer.save()